In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split


In [19]:
train_path = 'dataset/train/'
val_path = 'dataset/val/'
test_path = 'dataset/test/'

def load_images(folder_path):
    image_folder = folder_path + 'images/'
    mask_folder = folder_path + 'masks/'
    images = []
    masks = []
    for filename in os.listdir(image_folder):
        img = Image.open(os.path.join(image_folder, filename))
        mask = Image.open(os.path.join(mask_folder, filename.replace('.tif', '.png')))
        images.append(np.array(img))
        masks.append(np.array(mask))
    return np.array(images), np.array(masks)

train_images, train_masks = load_images(train_path)
val_images, val_masks = load_images(val_path)
test_images, test_masks = load_images(test_path)



In [21]:
class_dict = pd.read_csv('dataset/class_dict.csv')
label_dict = {}

for index, row in class_dict.iterrows():
    label_dict[(row['r'], row['g'], row['b'])] = row['name']



In [23]:
def masks_to_labels(masks, label_dict):
    label_map = {label: i for i, label in enumerate(label_dict.keys())}
    labels = []
    for mask in masks:
        mask = mask.astype('uint32')
        label = np.zeros((mask.shape[0], mask.shape[1]))
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                label[i,j] = label_map.get(tuple(mask[i,j,:]), len(label_dict)-1)
        labels.append(label)
    return np.array(labels)


train_labels = masks_to_labels(train_masks, label_dict)
val_labels = masks_to_labels(val_masks, label_dict)
test_labels = masks_to_labels(test_masks, label_dict)



In [25]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape

model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', padding='same', input_shape=(120,120,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(120*120, activation='softmax'))
model.add(Reshape((120,120)))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=32, epochs=10, validation_data=(val_images, val_labels))



Epoch 1/10
32/32 [==============================] - 22s 535ms/step - loss: 2875.4812 - accuracy: 0.0102 - val_loss: 1978.0280 - val_accuracy: 0.0072
Epoch 2/10
32/32 [==============================] - 16s 488ms/step - loss: 2872.4998 - accuracy: 0.0105 - val_loss: 1977.9896 - val_accuracy: 0.0090
Epoch 3/10
32/32 [==============================] - 17s 528ms/step - loss: 2872.5000 - accuracy: 0.0084 - val_loss: 1977.9645 - val_accuracy: 0.0087
Epoch 4/10
32/32 [==============================] - 16s 497ms/step - loss: 2872.4993 - accuracy: 0.0083 - val_loss: 1977.9645 - val_accuracy: 0.0086
Epoch 5/10
32/32 [==============================] - 16s 495ms/step - loss: 2872.4993 - accuracy: 0.0164 - val_loss: 1977.9645 - val_accuracy: 0.0145
Epoch 6/10
32/32 [==============================] - 16s 502ms/step - loss: 2872.4993 - accuracy: 0.0137 - val_loss: 1977.9645 - val_accuracy: 0.0084
Epoch 7/10
32/32 [==============================] - 16s 508ms/step - loss: 2872.4993 - accuracy: 0.0083 - 

In [51]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from deap import base, creator, tools, algorithms
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Conv2DTranspose
from tensorflow.keras.preprocessing.image import img_to_array, load_img


# Define the fitness function
def evaluate_model(model, train_data, val_data):
    train_images, train_labels = train_data
    val_images, val_labels = val_data
    history = model.fit(train_images, train_labels, batch_size=32, epochs=10, validation_data=(val_images, val_labels))
    return history.history['val_accuracy'][-1],

# Define the genetic algorithm parameters
POP_SIZE = 10
GEN_SIZE = 10
CXPB = 0.5
MUTPB = 0.2

# Create a fitness function with the appropriate number of input arguments
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Define the toolbox
toolbox = base.Toolbox()

# Define the initialization function for the individuals
def init_individual():
    return [np.random.randint(16, 64), np.random.uniform(0.0, 0.5), np.random.uniform(0.0, 0.5), np.random.uniform(0.0, 0.5)]

# Define the evaluation function for the individuals
def evaluate_individual(individual, train_data, val_data):
    model = Sequential()
    model.add(Conv2D(individual[0], (3,3), activation='relu', padding='same', input_shape=(120, 120, 3)))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(individual[0]*2, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(individual[0]*4, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(individual[1]))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(individual[2]))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(individual[3]))
    model.add(Dense(120*120*1, activation='softmax'))
    model.add(Reshape((120, 120, 1)))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return evaluate_model(model, train_data, val_data)

# Register the functions with the toolbox
toolbox.register("individual", tools.initIterate, creator.Individual, init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate_individual, train_data=(train_images, train_labels), val_data=(val_images, val_labels))

# Create the initial population
pop = toolbox.population(n=POP_SIZE)

In [52]:
# Run the genetic algorithm
# Define the number of generations
NGEN = 10
for g in range(NGEN):
    print(f"Generation {g+1}")
    # Select the next generation
    offspring = toolbox.select(pop, len(pop))
    
    # Apply crossover and mutation to create the offspring
    offspring = list(map(toolbox.clone, offspring))
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        if np.random.rand() < CXPB:
            toolbox.mate(ind1, ind2)
            del ind1.fitness.values
            del ind2.fitness.values
    
    for mutant in offspring:
        if np.random.rand() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    
    # Evaluate the fitness of the offspring
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    # Replace the population with the offspring
    pop[:] = offspring
    
    # Print the best individual of this generation
    best_ind = tools.selBest(pop, 1)[0]
    print(f"Best individual: {best_ind.fitness.values[0]}")
    


Generation 1
Epoch 1/10
32/32 [==============================] - 93s 2s/step - loss: 2956.3269 - accuracy: 0.0000e+00 - val_loss: 1977.9613 - val_accuracy: 0.0669
Epoch 2/10
32/32 [==============================] - 82s 3s/step - loss: 2872.5000 - accuracy: 0.0000e+00 - val_loss: 1977.9601 - val_accuracy: 0.0669
Epoch 3/10
32/32 [==============================] - 76s 2s/step - loss: 2872.5000 - accuracy: 0.0000e+00 - val_loss: 1977.9598 - val_accuracy: 0.0669
Epoch 4/10
32/32 [==============================] - 85s 3s/step - loss: 2872.4998 - accuracy: 0.0000e+00 - val_loss: 1977.9596 - val_accuracy: 0.0669
Epoch 5/10
32/32 [==============================] - 75s 2s/step - loss: 2872.4998 - accuracy: 0.0000e+00 - val_loss: 1977.9592 - val_accuracy: 0.0669
Epoch 6/10
32/32 [==============================] - 76s 2s/step - loss: 2872.4998 - accuracy: 0.0000e+00 - val_loss: 1977.9587 - val_accuracy: 0.0669
Epoch 7/10
32/32 [==============================] - 76s 2s/step - loss: 2872.4998 - acc

AttributeError: 'Individual' object has no attribute 'save'